In [1]:
import numpy as np

In [24]:
# Hot and cold numbered streams with properties such as heat capacities, starting/ending temperatures (actual and "shifted"),
# and temperature/enthalpy changes between their start and end points
class Stream:
    def __init__(self):
        self.num = 0
        self.cp = 0
        self.Ts = 0
        self.Tf = 0
        self.type = ''
        self.Ss = 0
        self.Sf = 0
        self.dT = 0
        self.dH = 0

# Temperature interval wherein a certain amount of heat should be transferred based purely on the thermodynamics
# of each stream operating within that interval. These hold many similar properties to streams since they are an abstract
# compilation of the properties of each stream acting in a given interval.
class Interval:
    def __init__(self):
        self.num = 0
        self.Ss = 0
        self.Sf = 0
        self.dT = 0
        self.dCp = 0
        self.dH = 0
        self.SD = ''

In [29]:
# Only user inputs required - arrays can be any length so long as all arrays match length/indices

streams = [1,2,3,4]  # Numerical list of streams
cp = [2,3,4,1.5]     # Each stream's specific heat capacity
Ts = [20,170,80,150] # Each stream's starting temperature
Tf = [135,60,140,30] # Each stream's ending temperature
dTmin = 10           # The minimum temperature difference desired between hot and cold streams in a countercurrent heat exchanger

In [55]:
intervals = []

# This loop assesses whether each stream is hot/cold (based on if its temperature decreases or increases) and then creates 
# an object possessing shifted temperatures as well as corresponding changes in shifted temperatures and enthalpy
for i in range(len(streams)):
    s = Stream()
    s.num = streams[i]
    s.cp = cp[i]
    s.Ts = Ts[i]
    s.Tf = Tf[i]
    
    if s.Ts < s.Tf:
        s.type = 'C'
        s.Ss = s.Ts + dTmin/2
        s.Sf = s.Tf + dTmin/2
        s.dT = s.Sf - s.Ss
        s.dH = s.cp*s.dT
        
    elif s.Ts > s.Tf:
        s.type = 'H'
        s.Ss = s.Ts - dTmin/2
        s.Sf = s.Tf - dTmin/2
        s.dT = s.Sf - s.Ss
        s.dH = s.cp*s.dT
        
    streams[i] = s

# Creating a basic list of shifte temperature intervals for a "heat cascade" that can be printed at completion 
temps = []
for i in range(0,len(streams)):
    temps.append(streams[i].Ss)
    temps.append(streams[i].Sf)

# Removing duplicate temperature interval values
temps = list(reversed(sorted(list(set(temps)))))
print('Interval Temperatures: ', temps)

enthalpies = []
cascade = []
adjCascade = []
intervals = []

# Creates interval objects in the same vein as those created for individual streams
for i in range(0,len(temps)-1):
    interval = Interval()
    interval.num = i+1
    interval.Ss = temps[i]
    interval.Sf = temps[i+1]
    interval.dT = temps[i]-temps[i+1]
    interval.dCp = 0
    
    # Determines which streams the interval contains for enthalpy calculations
    for j in streams:
        if j.type == 'H' and j.Ss >= interval.Ss and j.Sf <= interval.Sf:
            interval.dCp -= j.cp
            
        elif j.type == 'C' and j.Ss <= interval.Sf and j.Sf >= interval.Ss:
            interval.dCp += j.cp
    
    interval.dH = interval.dCp*interval.dT
    print('\nInterval %s begins at Ts = %s and ends at Tf = %s with dCp = %s and dH = %s'
         % (interval.num, interval.Ss, interval.Sf, interval.dCp, interval.dH))
    
    intervals.append(interval)
    enthalpies.append(interval.dH)
    
print('\nInterval Enthalpies: ', enthalpies)

H = 0
pinch = 0
adj = 0

#Determines the "pinch point" where heat exchange feasibility is most restricted
for i in range(0,len(enthalpies)):
    H += enthalpies[i]
    cascade.append(H)
    if H > 0:
        pinch = i+1
        adj = H
        print('Pinch point is at interval %s. Need %s more energy.' % (pinch, H))

# Prints the heat cascade intervals (including pinch)
print('Cascade: %s' % cascade)
        
# Adjusts and prints the heat cascade for pinch point heat requirements
for i in range(0,len(cascade)):
    adjCascade.append(cascade[i]-adj)
    
print('\nAdjusted Cascade: %s' % adjCascade)
print('\nPinch Point: %s deg C \nQhmin = %s \nQcmin = %s' % (temps[pinch], adj, abs(adjCascade[-1])))

Interval Temperatures:  [165.0, 145.0, 140.0, 85.0, 55.0, 25.0]

Interval 1 begins at Ts = 165.0 and ends at Tf = 145.0 with dCp = -3 and dH = -60.0

Interval 2 begins at Ts = 145.0 and ends at Tf = 140.0 with dCp = -0.5 and dH = -2.5

Interval 3 begins at Ts = 140.0 and ends at Tf = 85.0 with dCp = 1.5 and dH = 82.5

Interval 4 begins at Ts = 85.0 and ends at Tf = 55.0 with dCp = -2.5 and dH = -75.0

Interval 5 begins at Ts = 55.0 and ends at Tf = 25.0 with dCp = 0.5 and dH = 15.0

Interval Enthalpies:  [-60.0, -2.5, 82.5, -75.0, 15.0]
Pinch point is at interval 3. Need 20.0 more energy.
Cascade: [-60.0, -62.5, 20.0, -55.0, -40.0]

Adjusted Cascade: [-80.0, -82.5, 0.0, -75.0, -60.0]

Pinch Point: 85.0 deg C 
Qhmin = 20.0 
Qcmin = 60.0
